<a href="https://colab.research.google.com/github/bala-24g/Brain_tumour_classification/blob/main/Brain_tumour_accordingtopaper_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 pip install -U scikit-fuzzy

   ---------------------------------------- 0.0/920.8 kB ? eta -:--:--
   --------------------------------------- 920.8/920.8 kB 14.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install PyWavelets


   ---------------------------------------- 0.0/4.2 MB ? eta -:--:--
   --------- ------------------------------ 1.0/4.2 MB 12.7 MB/s eta 0:00:01
   ---------------------------------------- 4.2/4.2 MB 12.1 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import cv2
import skfuzzy as fuzz
import pywt
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt


In [ ]:


# Import necessary libraries
import os
import random
import cv2
import numpy as np
import pickle
from tqdm import tqdm
from sklearn.decomposition import PCA
import pywt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import skfuzzy as fuzz  # Now this should work

# Define necessary constants
TEST_DIR = 'C:\\Users\\nezes\\Documents\\Github_projects\\Brain_tumour\\Brain-Tumor-Classification-DataSet\\Testing'
TRAIN_DIR = 'C:\\Users\\nezes\\Documents\\Github_projects\\Brain_tumour\\Brain-Tumor-Classification-DataSet\\Training'
IMG_SIZE = 224

CATEGORIES = ["glioma_tumor", "meningioma_tumor", "no_tumor", "pituitary_tumor"]

# Creating training dataset
training_data = []

def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(TRAIN_DIR, category)
        class_num = CATEGORIES.index(category)
        for img in tqdm(os.listdir(path)):
            img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
            new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
            training_data.append([new_array, class_num])
    random.shuffle(training_data)

create_training_data()
print("Training data size:", len(training_data))

# Preparing training data
X_train = np.array([i[0] for i in training_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
Y_train = np.array([i[1] for i in training_data])

# Creating testing dataset
testing_data = []

def create_testing_data():
    for category in CATEGORIES:
        path = os.path.join(TEST_DIR, category)
        class_num = CATEGORIES.index(category)

        for img in tqdm(os.listdir(path)):
            img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
            new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
            testing_data.append([new_array, class_num])
    random.shuffle(testing_data)

create_testing_data()
print("Testing data size:", len(testing_data))

# Preparing testing data
X_test = np.array([i[0] for i in testing_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
Y_test = np.array([i[1] for i in testing_data])

# Step 3: Image Segmentation using Fuzzy C-means
def fuzzy_c_means_clustering(img, num_clusters=5):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_flat = img_gray.flatten()
    img_norm = img_flat / 255.0
    cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(img_norm.reshape(1, -1), num_clusters, 2, error=0.005, maxiter=1000)
    cluster_membership = np.argmax(u, axis=0)
    return cluster_membership.reshape(img_gray.shape)

# Apply segmentation to training and test data
X_train_segmented = np.array([fuzzy_c_means_clustering(img) for img in X_train])
X_test_segmented = np.array([fuzzy_c_means_clustering(img) for img in X_test])

# Step 4: Feature Extraction using Discrete Wavelet Transform (DWT)
def dwt_feature_extraction(img, wavelet='haar', level=3):
    coeffs = pywt.wavedec2(img, wavelet, level=level)
    coeff_arr, coeff_slices = pywt.coeffs_to_array(coeffs)
    return coeff_arr

# Extract features using DWT
X_train_features = np.array([dwt_feature_extraction(img) for img in X_train_segmented])
X_test_features = np.array([dwt_feature_extraction(img) for img in X_test_segmented])

# Step 5: Feature Reduction using PCA
pca = PCA(n_components=100)  # Adjust components based on your dataset
X_train_reduced = pca.fit_transform(X_train_features.reshape(X_train_features.shape[0], -1))
X_test_reduced = pca.transform(X_test_features.reshape(X_test_features.shape[0], -1))

# Step 6: Classification using DNN
def build_dnn_model(input_shape):
    model = Sequential()
    model.add(Dense(512, input_shape=(input_shape,), activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(len(CATEGORIES), activation='softmax'))  # Number of classes

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Build and train the DNN
input_shape = X_train_reduced.shape[1]
model = build_dnn_model(input_shape)

# Train the model
model.fit(X_train_reduced, Y_train, epochs=20, batch_size=32, validation_split=0.2)

# Step 7: Evaluate on Test Set
loss, accuracy = model.evaluate(X_test_reduced, Y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


100%|██████████| 827/827 [00:01<00:00, 447.24it/s]


Training data size: 2870


100%|██████████| 74/74 [00:00<00:00, 246.49it/s]


Testing data size: 394


c:\Users\nezes\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.3565 - loss: 2.9900 - val_accuracy: 0.4477 - val_loss: 1.2225
Epoch 2/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6056 - loss: 0.9704 - val_accuracy: 0.5174 - val_loss: 1.1657
Epoch 3/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7291 - loss: 0.6939 - val_accuracy: 0.5070 - val_loss: 1.2242
Epoch 4/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7863 - loss: 0.5686 - val_accuracy: 0.5801 - val_loss: 1.1514
Epoch 5/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8487 - loss: 0.4106 - val_accuracy: 0.5575 - val_loss: 1.2214
Epoch 6/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8951 - loss: 0.3239 - val_accuracy: 0.5819 - val_loss: 1.3738
Epoch 7/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9132 - loss: 0.2578 - val_accuracy: 0.5418 - val_loss: 1.5855
Epoch 8/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8995 - loss: 0.2735 - val_accuracy: 0.6202 - val_loss:

In [ ]:
# Clone the dataset from GitHub


# Import necessary libraries
import os
import random
import cv2
import numpy as np
import pickle
from tqdm import tqdm
from sklearn.decomposition import PCA
import pywt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import skfuzzy as fuzz

# Define necessary constants
TEST_DIR = '/content/Brain-Tumor-Classification-DataSet/Testing'
TRAIN_DIR = '/content/Brain-Tumor-Classification-DataSet/Training'
IMG_SIZE = 224
CATEGORIES = ["glioma_tumor", "meningioma_tumor", "no_tumor", "pituitary_tumor"]

# Creating training dataset
training_data = []

def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(TRAIN_DIR, category)
        class_num = CATEGORIES.index(category)
        for img in tqdm(os.listdir(path)):
            img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
            new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
            training_data.append([new_array, class_num])
    random.shuffle(training_data)

create_training_data()
print("Training data size:", len(training_data))

# Preparing training data
X_train = np.array([i[0] for i in training_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
Y_train = np.array([i[1] for i in training_data])

# Creating testing dataset
testing_data = []

def create_testing_data():
    for category in CATEGORIES:
        path = os.path.join(TEST_DIR, category)
        class_num = CATEGORIES.index(category)

        for img in tqdm(os.listdir(path)):
            img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
            new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
            testing_data.append([new_array, class_num])
    random.shuffle(testing_data)

create_testing_data()
print("Testing data size:", len(testing_data))

# Preparing testing data
X_test = np.array([i[0] for i in testing_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
Y_test = np.array([i[1] for i in testing_data])

# Step 3: Image Segmentation using Fuzzy C-means
def fuzzy_c_means_clustering(img, num_clusters=5):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_flat = img_gray.flatten()
    img_norm = img_flat / 255.0
    cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(img_norm.reshape(1, -1), num_clusters, 2, error=0.005, maxiter=1000)
    cluster_membership = np.argmax(u, axis=0)
    return cluster_membership.reshape(img_gray.shape)

# Apply segmentation to training and test data
X_train_segmented = np.array([fuzzy_c_means_clustering(img) for img in X_train])
X_test_segmented = np.array([fuzzy_c_means_clustering(img) for img in X_test])

# Step 4: Feature Extraction using Discrete Wavelet Transform (DWT)
def dwt_feature_extraction(img, wavelet='haar', level=3):
    coeffs = pywt.wavedec2(img, wavelet, level=level)
    coeff_arr, coeff_slices = pywt.coeffs_to_array(coeffs)
    return coeff_arr

# Extract features using DWT
X_train_features = np.array([dwt_feature_extraction(img) for img in X_train_segmented])
X_test_features = np.array([dwt_feature_extraction(img) for img in X_test_segmented])

# Step 5: Feature Reduction using PCA
pca = PCA(n_components=100)  # Adjust components based on your dataset
X_train_reduced = pca.fit_transform(X_train_features.reshape(X_train_features.shape[0], -1))
X_test_reduced = pca.transform(X_test_features.reshape(X_test_features.shape[0], -1))

# Step 6: Classification using DNN
def build_dnn_model(input_shape):
    model = Sequential()
    model.add(Dense(512, input_shape=(input_shape,), activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(len(CATEGORIES), activation='softmax'))  # Number of classes

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Build and train the DNN
input_shape = X_train_reduced.shape[1]
model = build_dnn_model(input_shape)

# Train the model
model.fit(X_train_reduced, Y_train, epochs=20, batch_size=32, validation_split=0.2)

# Step 7: Evaluate on Test Set
loss, accuracy = model.evaluate(X_test_reduced, Y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


In [ ]:
def fuzzy_c_means_segmentation(image, n_clusters=5, m=2, error=0.005, maxiter=1000):
    """
    Perform Fuzzy C-means clustering on a grayscale image.

    Parameters:
        image (numpy.ndarray): Grayscale image.
        n_clusters (int): Number of clusters.
        m (float): Fuzziness parameter.
        error (float): Stopping criterion.
        maxiter (int): Maximum number of iterations.

    Returns:
        segmented_image (numpy.ndarray): Segmented image with cluster labels.
    """
    # Flatten the image
    data = image.flatten().astype(np.float64)

    # Normalize data
    data_normalized = (data - data.min()) / (data.max() - data.min())

    # Transpose data for skfuzzy
    data_transposed = np.expand_dims(data_normalized, axis=0)

    # Fuzzy C-means
    cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(
        data_transposed, c=n_clusters, m=m, error=error, maxiter=maxiter, init=None)

    # Get the cluster membership for each data point
    cluster_membership = np.argmax(u, axis=0)

    # Reshape to original image shape
    segmented_image = cluster_membership.reshape(image.shape)

    return segmented_image


In [ ]:
def extract_dwt_features(image, wavelet='haar', level=3):
    """
    Extract DWT features from an image.

    Parameters:
        image (numpy.ndarray): Grayscale image.
        wavelet (str): Wavelet type.
        level (int): Decomposition level.

    Returns:
        features (numpy.ndarray): Flattened DWT coefficients.
    """
    coeffs = pywt.wavedec2(image, wavelet=wavelet, level=level)
    # Extract coefficients from all levels
    features = []
    for coeff in coeffs:
        if isinstance(coeff, tuple):
            for sub_coeff in coeff:
                features.extend(sub_coeff.flatten())
        else:
            features.extend(coeff.flatten())
    return np.array(features)


In [ ]:
def load_dataset(dataset_path, image_size=(256, 256)):
    """
    Load images and labels from the dataset directory.

    Parameters:
        dataset_path (str): Path to the dataset directory.
        image_size (tuple): Desired image size (width, height).

    Returns:
        images (list): List of preprocessed images.
        labels (list): Corresponding labels.
    """
    classes = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
    images = []
    labels = []

    for label, class_name in enumerate(classes):
        class_dir = os.path.join(dataset_path, class_name)
        for filename in os.listdir(class_dir):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.bmp')):
                img_path = os.path.join(class_dir, filename)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is not None:
                    img_resized = cv2.resize(img, image_size)
                    images.append(img_resized)
                    labels.append(label)
    return images, labels


In [ ]:
# Define dataset path
dataset_path =  'C:\\Users\\nezes\\Documents\\Github_projects\\Brain_tumour\\Brain-Tumor-Classification-DataSet\\Training'  # Replace with your dataset path

# Load images and labels
images, labels = load_dataset(dataset_path)

print(f"Total images: {len(images)}")
print(f"Total labels: {len(labels)}")


Total images: 2870
Total labels: 2870


In [ ]:
print(labels[0])
print(labels[1])


0
0


In [ ]:
# Initialize list to hold features
feature_list = []
label_list = []

for idx, img in enumerate(images):
    # Image Segmentation
    segmented = fuzzy_c_means_segmentation(img, n_clusters=5)

    # Extract the tumor part
    # Assuming the tumor corresponds to one of the clusters; you might need to adjust this
    # For simplicity, let's take the cluster with the highest intensity as tumor
    cluster_means = []
    for c in range(5):
        cluster_mean = img[segmented == c].mean() if np.any(segmented == c) else 0
        cluster_means.append(cluster_mean)
    tumor_cluster = np.argmax(cluster_means)
    tumor_segment = np.zeros_like(img)
    tumor_segment[segmented == tumor_cluster] = img[segmented == tumor_cluster]

    # Feature Extraction using DWT
    dwt_features = extract_dwt_features(tumor_segment, wavelet='haar', level=3)

    feature_list.append(dwt_features)
    label_list.append(labels[idx])

# Convert to numpy arrays
features = np.array(feature_list)
labels = np.array(label_list)

print(f"Extracted features shape: {features.shape}")
print(f"Labels shape: {labels.shape}")


Extracted features shape: (2870, 65536)
Labels shape: (2870,)


In [ ]:
# Initialize PCA to reduce to 1024 features (if original features > 1024)
pca = PCA(n_components=1024)
features_pca = pca.fit_transform(features)

print(f"PCA features shape: {features_pca.shape}")


PCA features shape: (2870, 1024)


In [ ]:
# One-hot encode labels for DNN
num_classes = 4
labels_categorical = to_categorical(labels, num_classes)

# Define stratified k-fold
skf = StratifiedKFold(n_splits=7, shuffle=True, random_state=42)


In [ ]:
# One-hot encode labels for DNN
num_classes = 4
labels_categorical = to_categorical(labels, num_classes)

# Define stratified k-fold
skf = StratifiedKFold(n_splits=7, shuffle=True, random_state=42)


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Initialize lists to store accuracies
dnn_accuracies = []
knn1_accuracies = []
knn3_accuracies = []
lda_accuracies = []

# Assuming you have defined `features_pca`, `labels`, and `labels_categorical`
skf = StratifiedKFold(n_splits=5)

for fold, (train_index, test_index) in enumerate(skf.split(features_pca, labels)):
    print(f"Fold {fold + 1}")

    # Split data
    X_train, X_test = features_pca[train_index], features_pca[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    y_train_cat, y_test_cat = labels_categorical[train_index], labels_categorical[test_index]

    # -------------------
    # Deep Neural Network
    # -------------------
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    # Compile model
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Train model
    history = model.fit(X_train, y_train_cat,
                        epochs=50,
                        batch_size=32,
                        verbose=0,
                        validation_data=(X_test, y_test_cat))

    # Evaluate model
    loss, acc = model.evaluate(X_test, y_test_cat, verbose=0)
    print(f"DNN Accuracy: {acc:.4f}")
    dnn_accuracies.append(acc)

    # -------------------
    # K-Nearest Neighbors (K=1 & K=3)
    # -------------------
    knn1 = KNeighborsClassifier(n_neighbors=1)
    knn1.fit(X_train, y_train)
    knn1_pred = knn1.predict(X_test)
    acc_knn1 = accuracy_score(y_test, knn1_pred)
    print(f"KNN-1 Accuracy: {acc_knn1:.4f}")
    knn1_accuracies.append(acc_knn1)

    knn3 = KNeighborsClassifier(n_neighbors=3)
    knn3.fit(X_train, y_train)
    knn3_pred = knn3.predict(X_test)
    acc_knn3 = accuracy_score(y_test, knn3_pred)
    print(f"KNN-3 Accuracy: {acc_knn3:.4f}")
    knn3_accuracies.append(acc_knn3)

    # -------------------
    # Linear Discriminant Analysis
    # -------------------
    lda = LinearDiscriminantAnalysis()
    lda.fit(X_train, y_train)
    lda_pred = lda.predict(X_test)
    acc_lda = accuracy_score(y_test, lda_pred)
    print(f"LDA Accuracy: {acc_lda:.4f}")
    lda_accuracies.append(acc_lda)

    print("-" * 30)


Fold 1


c:\Users\nezes\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


DNN Accuracy: 0.2875
KNN-1 Accuracy: 0.3659
KNN-3 Accuracy: 0.3432
LDA Accuracy: 0.4111
------------------------------
Fold 2


c:\Users\nezes\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


DNN Accuracy: 0.2875
KNN-1 Accuracy: 0.4861
KNN-3 Accuracy: 0.4547
LDA Accuracy: 0.4983
------------------------------
Fold 3


...

DNN Accuracy: 0.2875
KNN-1 Accuracy: 0.5314
KNN-3 Accuracy: 0.5261
LDA Accuracy: 0.5192
------------------------------
Fold 4


c:\Users\nezes\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


DNN Accuracy: 0.2875
KNN-1 Accuracy: 0.4930
KNN-3 Accuracy: 0.4721
LDA Accuracy: 0.4721
------------------------------
Fold 5


c:\Users\nezes\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


DNN Accuracy: 0.2875
KNN-1 Accuracy: 0.4390
KNN-3 Accuracy: 0.3955
LDA Accuracy: 0.4930
------------------------------


In [ ]:
print("Average Accuracies over 7 folds:")
print(f"DNN: {np.mean(dnn_accuracies):.4f} ± {np.std(dnn_accuracies):.4f}")
print(f"KNN-1: {np.mean(knn1_accuracies):.4f} ± {np.std(knn1_accuracies):.4f}")
print(f"KNN-3: {np.mean(knn3_accuracies):.4f} ± {np.std(knn3_accuracies):.4f}")
print(f"LDA: {np.mean(lda_accuracies):.4f} ± {np.std(lda_accuracies):.4f}")


Average Accuracies over 7 folds:
DNN: 0.2875 ± 0.0000
KNN-1: 0.4631 ± 0.0568
KNN-3: 0.4383 ± 0.0633
LDA: 0.4787 ± 0.0370
